# Short-Circuit Calculation according to IEC 60909

pandapower supports short-circuit calculations with the method of equivalent voltage source at the fault location according to IEC 60909. The pandapower short-circuit calculation supports the following elements:

- sgen (as motor or as full converter generator or as asynchronous machine or as doubly-fed asynchronous machine)
- gen (as synchronous generator)
- ext_grid
- line
- trafo
- trafo3w
- impedance

with the correction factors as defined in IEC 60909. Loads and shunts are neglected as per standard. The pandapower switch model is fully integrated into the short-circuit calculation. 

The following short-circuit currents can be calculated:
- ikss (Initial symmetrical short-circuit current)
- ip (short-circuit current peak)
- ith (equivalent thermal short-circuit current)

either as

 - symmetrical three-phase or
 - asymmetrical two-phase
 
short circuit current. Calculations are available for meshed as well as for radial networks. ip and ith are only implemented for short circuits far from synchronous generators.

In addition, short-circuit calculation with the superposition method is implemented. With this method, the pre-fault voltage is considered. The following differences to the worst-case calculation apply:
- transformer correction factor is not applied
- load, sgen are additionally modelled as shunt admittances (calculated based on their pre-fault currents)
- the grid must contain the results of a successful power flow calculation

The results for all elements and different short-circuit currents are tested against commercial software to ensure that correction factors are correctly applied. 

### Example Network

Here is a little example on how to use the short-circuit calculation. First, we create a simple open ring network with 4 buses, that are connected by one transformer and two lines with one open sectioning point. The network is fed by an external grid connection at bus 1:

<div style="background-color: white; display: inline-block; padding: 5px;">
    <img src="./example_network_sc.png">
</div>

In [ ]:
from pandapower.shortcircuit import calc_sc
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_ext_grid,
    create_lines,
    create_switch,
    create_transformer,
    create_sgen,
    create_gen
)
from pandapower.run import runpp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def ring_network():
    net = create_empty_network()
    b1 = create_bus(net, 220)
    b2 = create_bus(net, 110)
    b3 = create_bus(net, 110)
    b4 = create_bus(net, 110)
    create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)
    create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    _, l2, _ = create_lines(net, [b2, b3, b4], [b3, b4, b2], std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV" , length_km=[15, 12, 10])
    create_switch(net, b4, l2, closed=False, et="l")
    return net

## Symmetric Short-Circuit Calculation

### Maximum Short Circuit Currents
Now, we load the network and calculate the maximum short-circuit currents with the calc_sc function:

In [ ]:
net = ring_network()
calc_sc(net, case="max", ip=True, ith=True, branch_results=True)
net.res_bus_sc

where ikss is the initial short-circuit current, ip is the peak short-circuit current and ith is the thermal equivalent current.

For branches, the results are defined as the maximum current flows through that occurs for a fault at any bus in the network. The results are available seperately for lines:

In [ ]:
net.res_line_sc

and transformers:

In [ ]:
net.res_trafo_sc

### Minimum Short Circuit Currents

Minimum short-circuits can be calculated in the same way. However, we need to specify the end temperature of the lines after a fault as per standard first:

In [ ]:
net = ring_network()
net.line["endtemp_degree"] = 80
calc_sc(net, case="min", ith=True, ip=True, branch_results=True)
net.res_bus_sc

The branch results are now the minimum current flows through each branch:

In [ ]:
net.res_line_sc

In [ ]:
net.res_trafo_sc

### Asynchronous Motors

Asynchronous motors can be specified by creating a static generator of type "motor". For the short circuit impedance, an R/X ratio "rx" as well as the ratio between nominal current and short circuit current "k" has to be specified:

In [ ]:
net = ring_network()
create_sgen(net, 2, p_mw=0, sn_mva=0.5, k=1.2, rx=7., type="motor")
net

If we run the short-circuit calculation again, we can see that the currents increased due to the contribution of the inverteres to the short-circuit currents.

In [ ]:
calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

### Synchronous Generators

Synchronous generators can also be considered in the short-circuit calculation with the gen element. According to the standard, the rated power factor (cos$\varphi$) "cos_phi", rated voltage "vn_kv", rated apparent power "sn_kva" and subtransient resistance "rdss" and reactance "xdss" are necessary to calculate the short circuit impedance:

In [ ]:
net = ring_network()
create_gen(net, 2, p_mw=0, vm_pu=1.0, cos_phi=0.8, vn_kv=22, sn_mva=5, xdss_pu=0.2, rdss_ohm=0.005)
net

and run the short-circuit calculation again:

In [ ]:
calc_sc(net, case="max", ith=True, ip=True)
net.res_bus_sc

Once again, the short-circuit current increases due to the contribution of the generator. As can be seen in the warning, the values for peak and thermal equivalent short-circuit current will only be accurate for faults far from generators.

## Meshed Networks

The correction factors for aperiodic and thermal currents differ between meshed and radial networks. pandapower includes a meshing detection that automatically detects the meshing for each short-circuit location. Alternatively, the topology can be set to "radial" or "meshed" to circumvent the check and save calculation time.

We load the radial network and close the open sectioning point to get a closed ring network:

In [ ]:
net = ring_network()
net.switch.closed = True
calc_sc(net, topology="auto", ip=True, ith=True)
net.res_bus_sc

the network is automatically detected to be meshed and application factors are applied. This can be validated by setting the topology to radial and comparing the results:

In [ ]:
calc_sc(net, topology="radial", ip=True, ith=True)
net.res_bus_sc

If we look at the line results, we can see that the line currents are significantly smaller than the bus currents:

In [ ]:
calc_sc(net, topology="auto", ip=True, ith=True, branch_results=True)
net.res_line_sc

this is because the short-circuit current is split up on both paths of the ring, which is correctly considered by pandapower.

## Fault Impedance

It is also possible to specify a fault impedance in the short-circuit calculation:

In [ ]:
net = ring_network()
calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.)

which of course decreases the short-circuit currents:

In [ ]:
net.res_bus_sc

## Superposition method

We need to first execute the power flow calculation so that the grid has valid power flow results. We rely on the user to execute this step explicitly so that the user is fully aware that a power flow calculation is executed and also has control over all the relevant options for the power flow calculation.

Next, we pass the parameter use\_pre\_fault\_voltage = True to use the superposition method.

In [ ]:
runpp(net)
calc_sc(net, topology="radial", ip=True, ith=True, r_fault_ohm=1., x_fault_ohm=2.,
           branch_results=True, return_all_currents=True)

In [ ]:
net.res_bus_sc

In [ ]:
net.res_line_sc

The results above show the line results for the short-circuit calculation. The result table has a MultiIndex, which means that the results are shown for every line ("line" index column) and for every situation defined by a different fault location bus separately (index column "bus"). Note that not only the current magnitude values are available, but also the current angles, along with active and reactive power flows, and voltage magnitude and angle.

## Asymetrical Two-Phase Short-Circuit Calculation

All calculations above can be carried out for a two-phase short-circuit current in the same way by specifying "2ph" in the fault parameter:

In [ ]:
net = ring_network()
calc_sc(net, fault="2ph", ip=True, ith=True)
net.res_bus_sc

Two phase short-circuits are often used for minimum short-circuit calculations:

In [ ]:
net = ring_network()
net.line["endtemp_degree"] = 150
calc_sc(net, fault="2ph", case="min", ip=True, ith=True)
net.res_bus_sc

## Single Phase Short-Circuit Calculation


pandapower can also calculate single phase short-circuits. The ground fault however depends on the zero-sequence parameters of the network, which have to be added in order to calculate a single line to ground fault:

In [ ]:
net = ring_network()

#r/x ratio in zero sequence parameters
net.ext_grid["r0x0_max"] = 0.4
net.ext_grid["x0x_max"] = 1.0

#zero sequence line parameters
net.line["r0_ohm_per_km"] = 0.244
net.line["x0_ohm_per_km"] = 0.336
net.line["c0_nf_per_km"] = 2000

#transformer vector group, zero sequence short circuit voltage
#and zero sequence magnetizing impedance
net.trafo["vector_group"] = "Dyn"
net.trafo["vk0_percent"] = 5.
net.trafo["vkr0_percent"] = 0.4
net.trafo["mag0_percent"] = 1000
net.trafo["mag0_rx"] = 0.4
net.trafo["si0_hv_partial"] = 0.9

In [ ]:
calc_sc(net, fault="1ph")

In [ ]:
net.res_bus_sc